<a href="https://colab.research.google.com/github/rromerov/Proyecto_Integrador/blob/main/utils/molGeneratorTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clonar modelo de Hugging Face
! git clone https://huggingface.co/mrm8488/chEMBL_smiles_v1

Cloning into 'chEMBL_smiles_v1'...
remote: Enumerating objects: 49, done.
remote: Total 49 (delta 0), reused 0 (delta 0), pack-reused 49 (from 1)
Unpacking objects: 100% (49/49), 19.21 KiB | 1.60 MiB/s, done.
Filtering content: 100% (4/4), 275.08 MiB | 146.56 MiB/s, done.


In [ ]:
# Instalar librerias
! pip install datasets
! pip install transformers[torch]
! pip install accelerate -U
! pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, RobertaForCausalLM, pipeline
from datasets import Dataset
import numpy as np
import torch
from rdkit import Chem
from google.colab import drive

In [ ]:
# Cargar Google Drive al notebook
drive.mount('/content/drive')

# Cargar el dataset
ruta_archivo_bioactivity = '/content/drive/My Drive/Colab Notebooks/data/bioactivity_data_2class_pIC50.csv'

# Lee el archivo CSV en un DataFrame
df = pd.read_csv(ruta_archivo_bioactivity)

# Mostrar que se haya importado correctamente el archivo
df.head(5)

Mounted at /content/drive


,molecule_chembl_id,canonical_smiles,class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL2022564,CC[C@H](C)[C@H]1C(=O)N2CCC[C@H]2C(=O)O[C@H](C(...,active,828.130,5.1124,2.0,10.0,9.494850
1,CHEMBL3234200,CC[C@H](C)[C@H]1C(=O)N2CCC[C@H]2C(=O)O[C@H](C(...,active,814.103,4.8664,2.0,10.0,9.522879
2,CHEMBL3234201,CC[C@H](C)[C@H]1C(=O)N2CCC[C@H]2C(=O)O[C@H](C(...,active,842.157,5.5025,2.0,10.0,9.327902
3,CHEMBL3234202,CC[C@H](C)[C@H]1C(=O)N2CCC[C@H]2C(=O)O[C@H](C(...,active,828.130,5.1124,2.0,10.0,9.920819
4,CHEMBL3649592,CC1CCC(NCc2ccn(Cc3ccccc3)c(=O)c2O)CC1,inactive,326.440,3.2705,2.0,4.0,4.562249


In [ ]:
# Verificar si CUDA está disponible y definir el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Mostrar dataframe
df

,molecule_chembl_id,canonical_smiles,class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL2022564,CC[C@H](C)[C@H]1C(=O)N2CCC[C@H]2C(=O)O[C@H](C(...,active,828.130,5.1124,2.0,10.0,9.494850
1,CHEMBL3234200,CC[C@H](C)[C@H]1C(=O)N2CCC[C@H]2C(=O)O[C@H](C(...,active,814.103,4.8664,2.0,10.0,9.522879
2,CHEMBL3234201,CC[C@H](C)[C@H]1C(=O)N2CCC[C@H]2C(=O)O[C@H](C(...,active,842.157,5.5025,2.0,10.0,9.327902
3,CHEMBL3234202,CC[C@H](C)[C@H]1C(=O)N2CCC[C@H]2C(=O)O[C@H](C(...,active,828.130,5.1124,2.0,10.0,9.920819
4,CHEMBL3649592,CC1CCC(NCc2ccn(Cc3ccccc3)c(=O)c2O)CC1,inactive,326.440,3.2705,2.0,4.0,4.562249
...,...,...,...,...,...,...,...,...
10630,CHEMBL4636800,COc1ccc(-c2cnn3c(N)c(-c4ccccc4)c(NCCCN(C)C)nc2...,intermediate,446.555,4.0263,2.0,8.0,5.102373
10631,CHEMBL4644274,COc1ccc(-c2cnn3c(N)c(-c4ccccc4)c(NC4CC(C)(C)N(...,active,514.674,5.7260,2.0,8.0,6.004365
10632,CHEMBL4636056,CN1C(C)(C)CC(Nc2nc3c(-c4ccc(C(=O)N5C[C@@H]6C[C...,intermediate,579.749,5.3221,2.0,8.0,5.886057
10633,CHEMBL4643578,COC(=O)N1CCN(C(=O)c2ccc(-c3cnn4c(N)c(-c5ccccc5...,active,624.790,5.2330,2.0,9.0,6.050610


In [ ]:
df['text'] = df['canonical_smiles']

In [ ]:
# Cargar el modelo y el tokenizador
model_name = "mrm8488/chEMBL_smiles_v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Mover el modelo al dispositivo (GPU o CPU)
model.to(device)

# Convertir el DataFrame a un Dataset de Hugging Face
dataset = Dataset.from_pandas(df)

# Tokenizar los datos
def tokenize_function(examples):
    # Tokenizar el texto
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True)
    # Usar los mismos tokens como etiquetas para la tarea de Causal LM
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_datasets = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/501 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/28.7k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/96.9M [00:00<?, ?B/s]

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


Map:   0%|          | 0/10635 [00:00<?, ? examples/s]

In [ ]:
# Dividir el dataset en entrenamiento y validación
split_datasets = tokenized_datasets.train_test_split(test_size=0.1)
train_dataset = split_datasets['train']
eval_dataset = split_datasets['test']

In [ ]:
# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    logging_dir='./logs',  # Directorio para los registros
    logging_steps=100,  # Ajusta este número según tus necesidades
)

In [ ]:
# Inicializar el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

In [ ]:
# Entrenar el modelo
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.016600,0.010987
2,0.005600,0.005434
3,0.003200,0.004142
4,0.002200,0.003589
5,0.001800,0.003107
6,0.001600,0.002931
7,0.001200,0.002962
8,0.001100,0.002675
9,0.000900,0.002543
10,0.000800,0.002503


TrainOutput(global_step=8985, training_loss=0.017002917373004595, metrics={'train_runtime': 1875.7175, 'train_samples_per_second': 76.539, 'train_steps_per_second': 4.79, 'total_flos': 9640539719147520.0, 'train_loss': 0.017002917373004595, 'epoch': 15.0})

In [ ]:
# Evaluar el modelo
trainer.evaluate()

{'eval_loss': 0.002535122912377119,
 'eval_runtime': 4.776,
 'eval_samples_per_second': 222.779,
 'eval_steps_per_second': 14.028,
 'epoch': 15.0}

In [ ]:
# Crear carpeta dentro de google drive llamada results
! mkdir '/content/drive/My Drive/Colab Notebooks/data/results'

In [ ]:
# Guardar el modelo y el tokenizador
model.save_pretrained("/content/drive/My Drive/Colab Notebooks/data/results")
tokenizer.save_pretrained("/content/drive/My Drive/Colab Notebooks/data/results")

('/content/drive/My Drive/Colab Notebooks/results/tokenizer_config.json',
 '/content/drive/My Drive/Colab Notebooks/results/special_tokens_map.json',
 '/content/drive/My Drive/Colab Notebooks/results/vocab.json',
 '/content/drive/My Drive/Colab Notebooks/results/merges.txt',
 '/content/drive/My Drive/Colab Notebooks/results/added_tokens.json',
 '/content/drive/My Drive/Colab Notebooks/results/tokenizer.json')

In [ ]:
# Cargar el modelo y el tokenizador entrenados
model_name = "/content/drive/My Drive/Colab Notebooks/data/results"  # Directorio donde se guardaron los resultados del entrenamiento
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = RobertaForCausalLM.from_pretrained(model_name, is_decoder=True)  # Asegúrate de configurar el modelo como decodificador

# Crear el pipeline de fill-mask usando el modelo afinado
fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

# Función para validar moléculas SMILES
def is_valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None


In [ ]:
# Ejemplo de múltiples secuencias SMILES con máscaras en diferentes posiciones
smiles_list = [
    "CC(C)CN(CC(OP(=O)(O)O)C(Cc1ccccc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N)<mask>",
    "CC(C)CN(CC(OP(=O)(O)O)C(Cc1cc<mask>cc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N)cc1",
    "CC(C)CN(CC(OP(=O)(O)O)C(Cc1ccccc1)NC(=O)OC1<mask>COC1)S(=O)(=O)c1ccc(N)cc1"
]

# Generar secuencias completas para cada SMILES en la lista
for smile in smiles_list:
    results = fill_mask(smile)
    print(f"Original SMILES: {smile}")
    for result in results:
        sequence = result['sequence']
        # Quitar los tokens especiales si los hay
        sequence = sequence.replace('<s>', '').replace('</s>', '').strip()
        if is_valid_smiles(sequence):
            print(f"Valid SMILES: {sequence} | Score: {result['score']}")
        else:
            print(f"Invalid SMILES: {sequence} | Score: {result['score']}")
    print("-----")


Original SMILES: CC(C)CN(CC(OP(=O)(O)O)C(Cc1ccccc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N)<mask>
Invalid SMILES: CC(C)CN(CC(OP(=O)(O)O)C(Cc1ccccc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N)CS | Score: 0.5165514945983887
Invalid SMILES: CC(C)CN(CC(OP(=O)(O)O)C(Cc1ccccc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N)F | Score: 0.12074270099401474
Invalid SMILES: CC(C)CN(CC(OP(=O)(O)O)C(Cc1ccccc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N)CCO | Score: 0.09234333038330078
Invalid SMILES: CC(C)CN(CC(OP(=O)(O)O)C(Cc1ccccc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N)Cl | Score: 0.0388837531208992
Invalid SMILES: CC(C)CN(CC(OP(=O)(O)O)C(Cc1ccccc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N)) | Score: 0.024280305951833725
-----
Original SMILES: CC(C)CN(CC(OP(=O)(O)O)C(Cc1cc<mask>cc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N)cc1
Invalid SMILES: CC(C)CN(CC(OP(=O)(O)O)C(Cc1cc-cc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N)cc1 | Score: 0.3360162377357483
Invalid SMILES: CC(C)CN(CC(OP(=O)(O)O)C(Cc1cc2cc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N)cc1 | Score: 0.162947878241539
Invalid SMILES: C

[16:06:32] SMILES Parse Error: unclosed ring for input: 'CC(C)CN(CC(OP(=O)(O)O)C(Cc1ccccc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N)CS'
[16:06:32] SMILES Parse Error: unclosed ring for input: 'CC(C)CN(CC(OP(=O)(O)O)C(Cc1ccccc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N)F'
[16:06:32] SMILES Parse Error: unclosed ring for input: 'CC(C)CN(CC(OP(=O)(O)O)C(Cc1ccccc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N)CCO'
[16:06:32] SMILES Parse Error: unclosed ring for input: 'CC(C)CN(CC(OP(=O)(O)O)C(Cc1ccccc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N)Cl'
[16:06:32] SMILES Parse Error: extra close parentheses while parsing: CC(C)CN(CC(OP(=O)(O)O)C(Cc1ccccc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N))
[16:06:32] SMILES Parse Error: Failed parsing SMILES 'CC(C)CN(CC(OP(=O)(O)O)C(Cc1ccccc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N))' for input: 'CC(C)CN(CC(OP(=O)(O)O)C(Cc1ccccc1)NC(=O)OC1CCOC1)S(=O)(=O)c1ccc(N))'
[16:06:32] Can't kekulize mol.  Unkekulized atoms: 14 15 16 17 18
[16:06:32] SMILES Parse Error: unclosed ring for input: 'CC(C)CN(CC(OP(=O)(O)O)C(Cc1